## Libraries

In [2]:
!pip install psycopg2
import psycopg2 as ps
import pandas as pd
import requests
import time

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## API data extraction

In [15]:


#keys
API_KEY = 'your_key'

#the ID channel your're going ot extract info from, in my case is the National Geographic Channel 
CHANNEL_ID = 'UCnmlG_YzRYzWzJbW2oDn_ow'


In [16]:
def get_video_details(video_id):

    #collecting view, like, comment counts
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()

    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']

    return view_count, like_count, comment_count

In [17]:
def get_videos(df):
    pageToken = ""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken

        response = requests.get(url).json()
        time.sleep(1) #give it a second before starting the for loop
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, comment_count = get_video_details(video_id)

                df = df.append({'video_id':video_id,'video_title':video_title,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count,"comment_count":comment_count},ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [18]:
#main

#build our dataframe
df = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","comment_count"]) 

df = get_videos(df)

In [32]:
df.head()

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,bT-3O9M13QE,El Encanto de Nuestras Raíces: La Chamba | Nat...,2022-02-04,3578,190,3
1,jvfxW89Y95U,El Encanto de Nuestras Raíces: Aguadas | Natio...,2022-01-28,7102,334,10
2,MvQkNzIJCC8,El Encanto de Nuestras Raíces: Detrás de cámar...,2022-01-21,3544,124,7
3,zJ7AvnuZbjE,El Encanto de Nuestras Raíces: Zenú | National...,2022-01-21,8060,213,9
4,0QJjlXNBbwE,El Encanto de Nuestras Raíces: Wayúu | Nationa...,2022-01-14,11449,376,11


## Create DB

Before coding I created a database in AWS RDS:

Motor: PostgreSQL

Public access: Yes

Everything else stays as default.

If you want to use free tier, use Standard create to set the PostgreSQL version lower than version 13, and then choose Free tier in Templates.

In [20]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, port=port, database=dbname, user=username, password=password)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

In [28]:
# Create the schema for the table using the dataframe format
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
                    comment_count INTEGER NOT NULL
            )""")

    curr.execute(create_table_command)

def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date, view_count, like_count,comment_count) VALUES(%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)


def update_row(curr, video_id, video_title, view_count, like_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                comment_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
    curr.execute(query, vars_to_update)


def check_if_video_exists(curr, video_id): 
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")

    curr.execute(query, (video_id,))
    return curr.fetchone() is not None



def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE videos""")

    curr.execute(truncate_table)


def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['comment_count'])


def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count','like_count', 'comment_count'])
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']): # If video already exists then we will update
            update_row(curr,row['video_id'],row['video_title'],row['view_count'],row['like_count'],row['comment_count'])
        else: # The video doesn't exists so we will add it to a temporary df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)

    return tmp_df

In [22]:
#Main

#database credentials
host_name = 'xxxxxx.amazonaws.com'
dbname = 'xxxxxx'
port = '5432'
username = 'xxxxx' 
password = 'xxxxxx'
conn = None

#establish a connection to db
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

Connected!


In [23]:
#create table

create_table(curr)

In [24]:
#update data for existing videos

new_vid_df = update_db(curr,df)
conn.commit()

In [29]:
#insert new videos into db table

append_from_df_to_db(curr, new_vid_df)
conn.commit()


In [30]:
#view data in db table

curr.execute("SELECT * FROM VIDEOS")
print(curr.fetchall())

[('bT-3O9M13QE', 'El Encanto de Nuestras Raíces: La Chamba | National Geographic', datetime.date(2022, 2, 4), 3578, 190, 3), ('jvfxW89Y95U', 'El Encanto de Nuestras Raíces: Aguadas | National Geographic', datetime.date(2022, 1, 28), 7102, 334, 10), ('MvQkNzIJCC8', 'El Encanto de Nuestras Raíces: Detrás de cámara | National Geographic', datetime.date(2022, 1, 21), 3544, 124, 7), ('zJ7AvnuZbjE', 'El Encanto de Nuestras Raíces: Zenú | National Geographic', datetime.date(2022, 1, 21), 8060, 213, 9), ('0QJjlXNBbwE', 'El Encanto de Nuestras Raíces: Wayúu | National Geographic', datetime.date(2022, 1, 14), 11449, 376, 11), ('rAPjWaqip7A', 'Alerta Aeropuerto Colombia: Seducido por las redes', datetime.date(2021, 12, 28), 667808, 5649, 558), ('szmpGxU5R9M', 'Cuando Los Tiburones atacan: ¿Puede un virus causar el ataque? | National Geographic', datetime.date(2021, 12, 21), 7918, 172, 5), ('QIvXSY-2JsI', 'Cuando Los Tiburones Atacan: ¿Los atrae la sangre? | National Geographic', datetime.date(202